In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import sumolib
from utils import *
import json
from itertools import groupby


import warnings
warnings.filterwarnings("ignore")

### Create Mobility Demand

#### File paths

In [2]:
# od matrix path
od_matrix_path = "../data/od_matrix.npy"

# dict tile-edges path
dict_tile_edges_path = "../data/dict_tile_edges.json"

# dict mobility demand path
dict_mobility_demand_path = "../data/dict_mobility_demand.json"

# road network path
road_network_path = "../sumo_simulation_data/milan_road_network.net.xml"

# output filename
mobility_demand_save_path = "../sumo_simulation_data/mobility_demands/"

#### Load the OD matrix

In [3]:
od_matrix = np.load(od_matrix_path)

#### Load the dict tile-edges

In [4]:
with open(dict_tile_edges_path, 'r') as f:
    dict_tile_edges = json.load(f)

#### Load the road network

In [5]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

#### Create the Mobility Demand

In [6]:
n_veichles = 50
filename = mobility_demand_save_path+"mobility_demand"

In [7]:
od_pairs, departure_times, dict_mobility_demand = create_mobility_demand(n_veichles, dict_tile_edges, od_matrix, 
                                                                         road_network, filename, timing="uniform_range", 
                                                                         time_range=(0, 3601))

print("Created the mobility demand: "+mobility_demand_save_path+"mobility_demand.rou.xml")

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.93it/s]


Created the mobility demand: ../sumo_simulation_data/mobility_demands/mobility_demand.rou.xml


In [8]:
# save the dictionary that contains for each vehicle its corresponding trip (edge_start, edge_destination) and 
# the associated departure time

output_file = open(dict_mobility_demand_path, "w")
json.dump(dict_mobility_demand, output_file)
output_file.close()